In [1]:
import os
import subprocess
from pathlib import Path

platform = os.getenv('WORDSLAB_PLATFORM', 'UnknownLinux')
platform

'WindowsSubsystemForLinux'

In [2]:
# Get Ubuntu version
try:
    # Try reading from /etc/os-release first
    with open('/etc/os-release', 'r') as f:
        for line in f:
            if line.startswith('PRETTY_NAME='):
                os_version = line.split('=')[1].strip().strip('"')
                break
except:
    try:
        # Fallback to reading from /etc/issue
        with open('/etc/issue', 'r') as f:
            os_version = f.read().strip().split('\\')[0].strip()
    except:
        os_version = "Unknown"

os_version

'Ubuntu 24.04.1 LTS'

In [3]:
wordslab_version = os.getenv('WORDSLAB_VERSION', 'Unknown')
wordslab_version

'2025-01'

In [4]:
from importlib.metadata import version
import torch
import pynvml

apps_versions = { 'pytorch': version('torch') }

cpu_only_file = Path(os.getenv('WORDSLAB_NOTEBOOKS_ENV', '')) / '.cpu-only'
if not cpu_only_file.exists():
    try:
        if torch.cuda.is_available():
            apps_versions['python cuda'] = torch.version.cuda
    except:
        pass
    try:
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        driver_cuda_version = pynvml.nvmlSystemGetCudaDriverVersion_v2()
        apps_versions['gpu cuda'] = f"{driver_cuda_version//1000}.{(driver_cuda_version%1000)//10}"
        pynvml.nvmlShutdown()
    except:
        pass

apps_versions

{'pytorch': '2.5.1', 'python cuda': '12.4', 'gpu cuda': '12.8'}

### Get URL - Windows Subsystem for Linux

```
User req as a route parameter
=> req.url

Append port : 8880 - 8888
```

### Get URL - Jarvislabs

```
MACHINE_NAME=test-install2 (UI)
MACHINE_ID=261560 (API)

HOSTNAME=98708c9cbe44 (?)

reserved ports: 6006,7007

Native JUPYTERLAB
internal port: 8889
https://98708c261555.notebooks.jarvislabs.net/lab

APIs
https://98708c2615551.notebooks.jarvislabs.net/ => 6006
https://98708c2615552.notebooks.jarvislabs.net/ => Jupyterlab:8880
https://98708c2615553.notebooks.jarvislabs.net/ => VsCode:8881
https://98708c2615553.notebooks.jarvislabs.net/ => Ollama:8882
...
https://98708c26155510.notebooks.jarvislabs.net/ => Dashboard:8888
```

### Get URL - Runpod

```
RUNPOD_POD_ID=ebkhet9vq5flt7 (UI + used for URLs)

RUNPOD_PUBLIC_IP=69.30.85.49
RUNPOD_TCP_PORT_22=22047

RUNPOD_DC_ID=CA-MTL-1
RUNPOD_POD_HOSTNAME=ebkhet9vq5flt7-644112e4

HOSTNAME=5936ba7c007d (?)

Native JUPYTERLAB
internal port: 8888
https://ebkhet9vq5flt7-8888.proxy.runpod.net/lab

Http Services
https://ebkhet9vq5flt7-8880.proxy.runpod.net/
...
https://ebkhet9vq5flt7-8888.proxy.runpod.net/
```

### Get URL - Vast.ai

```
CONTAINER_ID=16855978 (UI)

HOSTNAME=9e254447c27d

PUBLIC_IPADDR=142.115.158.140
VAST_TCP_PORT_22=42790
VAST_TCP_PORT_8080=42702

Native JUPYTERLAB
internal port: 8080
https://142.115.158.140:42702/tree

Open ports
VAST_TCP_PORT_8880=42777
VAST_TCP_PORT_8881=42791
VAST_TCP_PORT_8882=42623
VAST_TCP_PORT_8883=42779
VAST_TCP_PORT_8884=42668
VAST_TCP_PORT_8885=42663
VAST_TCP_PORT_8886=42729
VAST_TCP_PORT_8887=42736
```

In [5]:
def get_root_urls(platform, req):

    if platform=="WindowsSubsystemForLinux" or platform=="UnknownLinux":
        
        dashboard_root_url = f"{req.url.scheme}://{req.url.host}"
        return [f"{dashboard_root_url}:{port}" for port in range(8880,8889)]
    
    elif platform=="Jarvislabs.ai":

        url_prefix, url_suffix = req.url.host.split('.', 1)
        return [f"{req.url.scheme}://{url_prefix}{i}.{url_suffix}" for i in range(2,11)]

    elif platform=="Runpod.io": 

        pod_id = os.getenv('RUNPOD_POD_ID')
        _, domain = req.url.host.split('.', 1)    
        return [f"{req.url.scheme}://{pod_id}-{port}.{domain}" for port in range(8880,8889)]

    elif platform=="Vast.ai": 

        public_ip = os.getenv('PUBLIC_IPADDR')
        external_ports = [os.getenv(f'VAST_TCP_PORT_{port}') for port in range(8880,8889)]
        return [f"{req.url.scheme}://{public_ip}:{ext_port}" for ext_port in external_ports]

In [6]:
from types import SimpleNamespace

reqWSL = lambda: None
reqWSL.url = lambda: None
reqWSL.url.scheme = "http"
reqWSL.url.host = "127.0.0.1"

reqJarvis = lambda: None
reqJarvis.url = lambda: None
reqJarvis.url.scheme = "https"
reqJarvis.url.host = "98708c261555.notebooks.jarvislabs.net"

reqRunpod = lambda: None
reqRunpod.url = lambda: None
reqRunpod.url.scheme = "https"
reqRunpod.url.host = "ebkhet9vq5flt7-8888.proxy.runpod.net"

os.environ["RUNPOD_POD_ID"] = "ebkhet9vq5flt7"

reqVast = lambda: None
reqVast.url = lambda: None
reqVast.url.scheme = "https"
reqVast.url.host = "142.115.158.140"

os.environ["PUBLIC_IPADDR"] = "142.115.158.140"
os.environ["VAST_TCP_PORT_8880"] = "42777"
os.environ["VAST_TCP_PORT_8881"] = "42791"
os.environ["VAST_TCP_PORT_8882"] = "42623"
os.environ["VAST_TCP_PORT_8883"] = "42779"
os.environ["VAST_TCP_PORT_8884"] = "42668"
os.environ["VAST_TCP_PORT_8885"] = "42663"
os.environ["VAST_TCP_PORT_8886"] = "42729"
os.environ["VAST_TCP_PORT_8887"] = "42736"

In [7]:
get_root_urls("WindowsSubsystemForLinux", reqWSL)

['http://127.0.0.1:8880',
 'http://127.0.0.1:8881',
 'http://127.0.0.1:8882',
 'http://127.0.0.1:8883',
 'http://127.0.0.1:8884',
 'http://127.0.0.1:8885',
 'http://127.0.0.1:8886',
 'http://127.0.0.1:8887',
 'http://127.0.0.1:8888']

In [8]:
get_root_urls("Jarvislabs.ai", reqJarvis)

['https://98708c2615552.notebooks.jarvislabs.net',
 'https://98708c2615553.notebooks.jarvislabs.net',
 'https://98708c2615554.notebooks.jarvislabs.net',
 'https://98708c2615555.notebooks.jarvislabs.net',
 'https://98708c2615556.notebooks.jarvislabs.net',
 'https://98708c2615557.notebooks.jarvislabs.net',
 'https://98708c2615558.notebooks.jarvislabs.net',
 'https://98708c2615559.notebooks.jarvislabs.net',
 'https://98708c26155510.notebooks.jarvislabs.net']

In [9]:
get_root_urls("Runpod.io", reqRunpod)

['https://ebkhet9vq5flt7-8880.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8881.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8882.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8883.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8884.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8885.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8886.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8887.proxy.runpod.net',
 'https://ebkhet9vq5flt7-8888.proxy.runpod.net']

In [10]:
get_root_urls("Vast.ai", reqVast)

['https://142.115.158.140:42777',
 'https://142.115.158.140:42791',
 'https://142.115.158.140:42623',
 'https://142.115.158.140:42779',
 'https://142.115.158.140:42668',
 'https://142.115.158.140:42663',
 'https://142.115.158.140:42729',
 'https://142.115.158.140:42736',
 'https://142.115.158.140:None']

In [11]:
import psutil
import subprocess
from dataclasses import dataclass

@dataclass 
class CPUMetrics:
    cpu_model: str
    cpu_cores: int
    cpu_usage: float
    ram_total: float  # GB
    ram_used: float   # GB

def get_cpu_metrics() -> CPUMetrics:
    # Get CPU model
    try:
        with open('/proc/cpuinfo', 'r') as f:
            for line in f:
                if line.startswith('model name'):
                    cpu_model = line.split(':')[1].strip()
                    break
    except:
        cpu_model = "Unknown"

    cpu_frequency = int(round(psutil.cpu_freq().current, -2))
    
    # Get number of logical cores
    cpu_cores = psutil.cpu_count(logical=True)
    
    # Get CPU usage percentage
    cpu_usage = psutil.cpu_percent()
    
    # Get RAM info in GB
    ram = psutil.virtual_memory()
    ram_total = ram.total / (1024**3)  # Convert bytes to GB
    ram_used = ram.used / (1024**3)    # Convert bytes to GB
    
    return CPUMetrics(
        cpu_model=f"{cpu_model} {cpu_frequency} MHz",
        cpu_cores=cpu_cores,
        cpu_usage=cpu_usage,
        ram_total=ram_total,
        ram_used=ram_used
    )

In [12]:
get_cpu_metrics()

CPUMetrics(cpu_model='13th Gen Intel(R) Core(TM) i7-13700K 3400 MHz', cpu_cores=24, cpu_usage=0.8, ram_total=31.036624908447266, ram_used=1.8710403442382812)

In [44]:
from dataclasses import dataclass
import pynvml

@dataclass
class GPUMetrics:
    gpu_model: str
    gpu_usage: float  # percentage
    vram_total: float # GB
    vram_used: float  # GB

def get_gpu_metrics(gpu_id=0):
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(gpu_id)

    gpu_model = pynvml.nvmlDeviceGetName(handle)
    utilization = pynvml.nvmlDeviceGetUtilizationRates(handle)
    gpu_usage = utilization.gpu
    vram_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    vram_total = vram_info.total / 1024**3  # Convert bytes to GB
    vram_used = vram_info.used / 1024**3   # Convert bytes to GB

    pynvml.nvmlShutdown()
    
    metrics = GPUMetrics(
        gpu_model=gpu_model,
        gpu_usage=gpu_usage,
        vram_total=vram_total,
        vram_used=vram_used
    )

    return metrics

# Example usage:
metrics = get_gpu_metrics()
print(metrics)

GPUMetrics(gpu_model='NVIDIA GeForce RTX 4090', gpu_usage=68, vram_total=23.98828125, vram_used=23.87488555908203)


In [75]:
import os

wordslab_home = os.getenv('WORDSLAB_HOME')
wordslab_workspace = os.getenv('WORDSLAB_WORKSPACE')
wordslab_models = os.getenv('WORDSLAB_MODELS')
wordslab_paths = { "/", wordslab_home, wordslab_workspace, wordslab_models }

In [76]:
wordslab_paths

{'/', '/home', '/home/models', '/home/workspace'}

In [106]:
import subprocess

def get_directory_size(path):
   # Use the 'du' command to get the total size of the directory
    result = subprocess.run(['du', '-s', '--block-size=1', path], capture_output=True, text=True, check=True)
    # The output is in the format: <size>\t<path>
    size = int(result.stdout.split('\t')[0])
    return size

In [125]:
worsdlab_paths_sizes = {path:get_directory_size(path) for path in [wordslab_home, wordslab_workspace, wordslab_models]}
if wordslab_workspace.startswith(wordslab_home):
    worsdlab_paths_sizes[wordslab_home] -= worsdlab_paths_sizes[wordslab_workspace]
if wordslab_models.startswith(wordslab_home):
    worsdlab_paths_sizes[wordslab_home] -= worsdlab_paths_sizes[wordslab_models]

In [126]:
worsdlab_paths_sizes

{'/home': 20367511552,
 '/home/workspace': 2152120320,
 '/home/models': 413459570688}

In [83]:
import psutil
mountpoints = {part.mountpoint:part.device for part in set(psutil.disk_partitions(all=True)) if part.fstype in {'ext4','xfs','fuse','overlay'} and not any(substring in part.mountpoint for substring in ["/mnt/", "nvidia", ".so"])}

In [84]:
mountpoints

{'/home/workspace': '/dev/sdc',
 '/home/models': '/dev/sde',
 '/var/lib/docker': '/dev/sdd',
 '/usr/lib/wsl/lib': '',
 '/usr/lib/modules/5.15.167.4-microsoft-standard-WSL2': '',
 '/': '/dev/sdd'}

In [91]:
def map_paths_to_devices(wordslab_paths, mountpoints):
    wordslab_paths_devices = {}
    sorted_mountpoints = sorted(mountpoints.keys(), key=len, reverse=True)
    for path in wordslab_paths:
        for mountpoint in sorted_mountpoints:
            if path.startswith(mountpoint):
                device = mountpoints[mountpoint]
                if device not in wordslab_paths_devices:
                    wordslab_paths_devices[device] = [path]
                else:
                    wordslab_paths_devices[device].append(path)
                break
    return wordslab_paths_devices 

In [92]:
wordslab_paths_devices = map_paths_to_devices(wordslab_paths, mountpoints)
wordslab_paths_devices 

{'/dev/sdd': ['/', '/home'],
 '/dev/sdc': ['/home/workspace'],
 '/dev/sde': ['/home/models']}

In [104]:
wordslab_devices_usage = {item[0]:psutil.disk_usage(item[1][0]) for item in wordslab_paths_devices.items()}

In [105]:
wordslab_devices_usage

{'/dev/sdd': sdiskusage(total=1081101176832, used=26948464640, free=999160356864, percent=2.6),
 '/dev/sdc': sdiskusage(total=1081101176832, used=2160680960, free=1023948140544, percent=0.2),
 '/dev/sde': sdiskusage(total=1081101176832, used=413468131328, free=612640690176, percent=40.3)}

In [147]:
root_size = 0
for device in wordslab_paths_devices:
    if '/' in wordslab_paths_devices[device]:
        for path in wordslab_paths_devices[device]:
            if path == '/':
                root_size += wordslab_devices_usage[device].used
            else:
                root_size -= worsdlab_paths_sizes[path]
root_size/1024**3

6.128990173339844

In [127]:
from pathlib import Path

wordslab_windows_home = (Path(wordslab_home) / ".WORDSLAB_WINDOWS_HOME").read_text().strip()
wordslab_windows_workspace = (Path(wordslab_workspace) / ".WORDSLAB_WINDOWS_WORKSPACE").read_text().strip()
wordslab_windows_models = (Path(wordslab_models) / ".WORDSLAB_WINDOWS_MODELS").read_text().strip()

In [128]:
wordslab_windows_home,wordslab_windows_workspace,wordslab_windows_models

('c:\\wordslab\\virtual-machines\\wordslab-notebooks',
 'C:\\wordslab\\virtual-machines\\wordslab-workspace',
 'd:\\wordslab\\virtual-machines\\wordslab-models')

In [137]:
def windows_path_to_linux_vm_file(windows_path):
    # Replace backslashes with forward slashes
    windows_path = windows_path.replace('\\', '/')
    # Replace the drive letter with /mnt/<drive>
    if len(windows_path) > 1 and windows_path[1] == ':':
        drive_letter = windows_path[0].lower()
        drive_path = f"/mnt/{drive_letter}"
        linux_file = Path(f"{drive_path}{windows_path[2:]}") / "ext4.vhdx"
    return (windows_path[:2], psutil.disk_usage(drive_path), f"{windows_path}\\ext4.vhdx", linux_file.stat().st_size) 

In [138]:
wordslab_windows_home_vm_file = windows_path_to_linux_vm_file(wordslab_windows_home)
wordslab_windows_workspace_vm_file = windows_path_to_linux_vm_file(wordslab_windows_workspace)
wordslab_windows_models_vm_file = windows_path_to_linux_vm_file(wordslab_windows_models)

In [139]:
wordslab_windows_home_vm_file, wordslab_windows_workspace_vm_file, wordslab_windows_models_vm_file

(('c:',
  sdiskusage(total=2047178960896, used=142562906112, free=1904616054784, percent=7.0),
  'c:/wordslab/virtual-machines/wordslab-notebooks\\ext4.vhdx',
  30211571712),
 ('C:',
  sdiskusage(total=2047178960896, used=142562906112, free=1904616054784, percent=7.0),
  'C:/wordslab/virtual-machines/wordslab-workspace\\ext4.vhdx',
  2858418176),
 ('d:',
  sdiskusage(total=4000768323584, used=1041897127936, free=2958871195648, percent=26.0),
  'd:/wordslab/virtual-machines/wordslab-models\\ext4.vhdx',
  413483925504))

### Jarvislabs
- / -> overlay size No
- /home -> /dev/rdb0 size Yes

import psutil
{part.mountpoint:part.device for part in set(psutil.disk_partitions(all=True)) if part.fstype in {'ext4','xfs','fuse','overlay'} and not "/mnt/" in part.mountpoint and not "nvidia" in part.mountpoint and not ".so" in part.mountpoint}

{'/etc/hostname': '/dev/nvme0n1p3',
 '/etc/resolv.conf': '/dev/nvme0n1p3',
 '/home': '/dev/rbd0',
 '/usr/sbin/docker-init': '/dev/nvme0n1p3',
 '/': 'overlay',
 '/etc/hosts': '/dev/nvme0n1p3'}

!df -h /

Filesystem      Size  Used Avail Use% Mounted on
overlay         1.9T  895G  886G  51% /

!df -h /home

Filesystem      Size  Used Avail Use% Mounted on
/dev/rbd1        20G  177M   20G   1% /home

!du -sh /* | sort -h

0	/bin
0	/dev
0	/lib
0	/lib32
0	/lib64
0	/libx32
0	/sbin
0	/sys
4.0K	/boot
4.0K	/docker-entrypoint.sh
4.0K	/media
4.0K	/mnt
4.0K	/srv
4.0K	/tmp
4.0K	/workspace
8.0K	/cuda-keyring_1.1-1_all.deb
12K	/proc
20K	/NGC-DL-CONTAINER-LICENSE
92K	/run
2.0M	/home
2.3M	/etc
70M	/var
1.4G	/opt
6.4G	/usr
9.2G	/root

### Runpod
- / -> overlay size ?
- /workspace -> fuse size KO

import psutil
{part.mountpoint:part.device for part in set(psutil.disk_partitions(all=True)) if part.fstype in {'ext4','xfs','fuse','overlay'} and not "/mnt/" in part.mountpoint and not "nvidia" in part.mountpoint and not ".so" in part.mountpoint}

{'/usr/sbin/docker-init': '/dev/nvme3n1p3',
 '/workspace': 'mfs\\043eu-cz-1.runpod.net:9421',
 '/etc/hosts': '/dev/md0p1',
 '/etc/resolv.conf': '/dev/md0p1',
 '/etc/hostname': '/dev/md0p1',
 '/': 'overlay'}

!df -h /workspace

Filesystem                   Size  Used Avail Use% Mounted on
mfs#eu-cz-1.runpod.net:9421  234T  110T  125T  47% /workspace

!du -sh /* | sort -h

0	/bin
0	/boot
0	/dev
0	/home
0	/lib
0	/lib32
0	/lib64
0	/libx32
0	/media
0	/mnt
0	/sbin
0	/srv
0	/sys
0	/tmp
4.0K	/start.sh
8.0K	/cuda-keyring_1.0-1_all.deb
8.0K	/jupyter.log
12K	/proc
16K	/run
20K	/NGC-DL-CONTAINER-LICENSE
999K	/workspace
1.9M	/etc
2.6M	/get-pip.py
15M	/var
44M	/root
918M	/opt
12G	/usr


### Vast
- / -> overlay size OK

{'/etc/hosts': '/dev/md127p1',
 '/': 'overlay',
 '/etc/resolv.conf': '/dev/md127p1',
 '/etc/hostname': '/dev/md127p1'}

!du -sh /* | sort -h

0	/bin
0	/boot
0	/dev
0	/lib
0	/lib32
0	/lib64
0	/libx32
0	/media
0	/mnt
0	/sbin
0	/srv
0	/sys
4.0K	/Untitled.ipynb
8.0K	/cuda-keyring_1.0-1_all.deb
12K	/proc
16K	/workspace
20K	/NGC-DL-CONTAINER-LICENSE
20K	/run
68K	/home
424K	/tmp
2.3M	/etc
21M	/root
108M	/var
1.8G	/opt
5.0G	/venv
12G	/usr

!df -h /

 Filesystem      Size  Used Avail Use% Mounted on
overlay          16G  2.0M   16G   1% /

In [140]:
jarvislabs_mountpoints = {'/etc/hostname': '/dev/nvme0n1p3', '/etc/resolv.conf': '/dev/nvme0n1p3', '/home': '/dev/rbd0', '/usr/sbin/docker-init': '/dev/nvme0n1p3', '/': 'overlay', '/etc/hosts': '/dev/nvme0n1p3'}
runpod_mountpoints = {'/usr/sbin/docker-init': '/dev/nvme3n1p3', '/workspace': 'mfs\\043eu-cz-1.runpod.net:9421', '/etc/hosts': '/dev/md0p1', '/etc/resolv.conf': '/dev/md0p1', '/etc/hostname': '/dev/md0p1', '/': 'overlay'}
vast_mountpoints = {'/etc/hosts': '/dev/md127p1', '/': 'overlay', '/etc/resolv.conf': '/dev/md127p1', '/etc/hostname': '/dev/md127p1'}

In [141]:
def map_paths_to_devices(wordslab_paths, mountpoints):
    wordslab_paths_devices = {}
    sorted_mountpoints = sorted(mountpoints.keys(), key=len, reverse=True)
    for path in wordslab_paths:
        for mountpoint in sorted_mountpoints:
            if path.startswith(mountpoint):
                wordslab_paths_devices[path] = mountpoints[mountpoint]
                break
    return  wordslab_paths_devices   

In [142]:
map_paths_to_devices(wordslab_paths, wsl_mountpoints)

{'/': '/dev/sdd',
 '/home': '/dev/sdd',
 '/home/workspace': '/dev/sdc',
 '/home/models': '/dev/sde'}

In [143]:
map_paths_to_devices(wordslab_paths, jarvislabs_mountpoints)

{'/': 'overlay',
 '/home': '/dev/rbd0',
 '/home/workspace': '/dev/rbd0',
 '/home/models': '/dev/rbd0'}

In [144]:
map_paths_to_devices({'/', '/workspace', '/workspace/models', '/workspace/workspace'}, runpod_mountpoints)

{'/': 'overlay',
 '/workspace/workspace': 'mfs\\043eu-cz-1.runpod.net:9421',
 '/workspace': 'mfs\\043eu-cz-1.runpod.net:9421',
 '/workspace/models': 'mfs\\043eu-cz-1.runpod.net:9421'}

In [145]:
map_paths_to_devices(wordslab_paths, vast_mountpoints)

{'/': 'overlay',
 '/home': 'overlay',
 '/home/workspace': 'overlay',
 '/home/models': 'overlay'}